In [1]:
import numpy as np
import h5py as h5

In [64]:
h5_name = 'baikal_multi_0523_flat_pureMC_h5s2_norm.h5'
path_to_data = './data/'

In [65]:
path_to_h5 = path_to_data + h5_name

In [156]:
class info_Baikal_HDF5:
    def __init__(self, h5_name, path_to_data = './data/'):
        self.path = path_to_data + h5_name
        #self.nums_in_train, self.nums_in_test, self.nums_in_val = None, None, None
        self.nums_dict = {'train': None, 'test': None, 'val': None}
        self.masks_dict = {'train': None, 'test': None, 'val': None}

    def collect_info(self, regime):
        with h5.File(self.path, 'r') as hf:
            ids = hf[regime+'/ev_ids_corr/data']
            print('ids_collected')
            l_nuatm, l_nue2, l_mu = [], [], []
            for i in ids:
                l_nuatm.append(i.startswith(b'nuatm'))
                l_nue2.append(i.startswith(b'nue2'))
                l_mu.append(i.startswith(b'mu'))
            num_mu = sum(l_mu)
            print('mu_collected')
            num_atm_nu = sum(l_nuatm)
            print('nu_atm_collected')
            num_e2_nu = sum(l_nue2)
            print('nu_e2_collected')
            self.nums_dict[regime] = {'mu': num_mu, 'nu_atm': num_atm_nu, 'nu_e2': num_e2_nu}  
            self.masks_dict[regime] = {'mu': l_mu, 'nu_atm': l_nuatm, 'nu_e2': l_nue2}

    def get_info_about(self, regime, return_masks = False):
        if not return_masks:
            return self.nums_dict[regime]
        else:
            return self.nums_dict[regime], self.masks_dict[regime]

In [ ]:
#Preparation to make small h5, ~30 min
info = info_Baikal_HDF5(h5_name)
for r in ['train','test','val']:
    print(r)
    info.collect_info(r)

In [160]:
# makes small h5 from bigger one
path_to_h5_small = path_to_data + h5_name[0:-3] + '_small.h5'

with h5.File(path_to_h5_small, 'w') as fs:
    with h5.File(path_to_h5, 'r') as hf:
        fs.create_dataset('norm_param/mean', data = hf['norm_param/mean'])
        fs.create_dataset('norm_param/std', data = hf['norm_param/std'])
        for r in ['train','test','val']:
            #set size of new file to 1/10 of initial one
            new_length = hf[r+'/ev_ids_corr/data'].shape[0]//10
            print(new_length)
            if r == 'train':
                start = 0
                stop = start+new_length
                start_data = hf[r+'/ev_starts/data'][start]
                stop_data = hf[r+'/ev_starts/data'][stop]
                
                ev_starts = hf[r+'/ev_starts/data'][start : stop]
                ids = hf[r+'/ev_ids_corr/data'][start : stop]
                data = hf[r+'/data/data'][start_data:stop_data]
                
                fs.create_dataset(r+'/ev_starts/data', data = ev_starts)
                fs.create_dataset(r+'/data/data', data = data)
                fs.create_dataset(r+'/ev_ids_corr/data', data = ids)
                
            else:
                start1 = 0
                stop1 = start1+new_length//2
                start_data1 = hf[r+'/ev_starts/data'][start1]
                stop_data1 = hf[r+'/ev_starts/data'][stop1]
                
                #finding where muons end
                start2 = info.nums_dict[r]['mu']
                stop2 = start2+new_length//4
                start_data2 = hf[r+'/ev_starts/data'][start2]
                stop_data2 = hf[r+'/ev_starts/data'][stop2]
                
                #finding where atmosheric neutrinos end
                start3 = info.nums_dict[r]['mu']+info.nums_dict[r]['nu_atm']
                stop3 = start3+new_length//4
                start_data3 = hf[r+'/ev_starts/data'][start3]
                stop_data3 = hf[r+'/ev_starts/data'][stop3]
                
                ev_starts = np.concatenate([hf[r+'/ev_starts/data'][start1:stop1],
                                            hf[r+'/ev_starts/data'][start2:stop2],
                                            hf[r+'/ev_starts/data'][start3:stop3]],
                                           axis = 0)

                ids = np.concatenate([hf[r+'/ev_ids_corr/data'][start1:stop1],
                                      hf[r+'/ev_ids_corr/data'][start2:stop2],
                                      hf[r+'/ev_ids_corr/data'][start3:stop3]],
                                     axis = 0)
                
                data = np.concatenate([hf[r+'/data/data'][start_data1:stop_data1],
                                      hf[r+'/data/data'][start_data2:stop_data2],
                                      hf[r+'/data/data'][start_data3:stop_data3]],
                                     axis = 0)
                
                fs.create_dataset(r+'/ev_starts/data', data = ev_starts)
                fs.create_dataset(r+'/data/data', data = data)
                fs.create_dataset(r+'/ev_ids_corr/data', data = ids)

555614
46525
2234582
